<a href="https://colab.research.google.com/github/moosakazim12/SQL-Practice/blob/main/SQL_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Question (SELF JOIN)
# How can you output a list of all members who have recommended another member? Ensure that there are no duplicates in the list, and that results are ordered by (surname, firstname).

SELECT DISTINCT
       m.firstname,
       m.surname
FROM cd.members m
INNER JOIN cd.members r
  ON m.memid = r.recommendedby
ORDER BY m.surname, m.firstname;


print("Query executed successfully.")
eeeee

In [ ]:
# Question
# How can you produce a list of the start times for bookings by members named 'David Farrell'?
# I have taken tables and question from https://pgexercises.com/questions/joins/simplejoin.html

SELECT starttime
FROM cd.members AS m
INNER JOIN cd.bookings AS b
USING(memid)
WHERE m.firstname = 'David' AND m.surname = 'Farrell'

In [ ]:
# Question
# How can you output a list of all members, including the individual who recommended them (if any)? Ensure that results are ordered by (surname, firstname).
SELECT m1.firstname AS memfname, m1.surname AS memsname,
m2.firstname AS recfname, m2.surname AS recsname
FROM cd.members AS m1
LEFT JOIN cd.members AS m2
 ON m2.memid = m1.recommendedby
ORDER BY memsname,memfname

In [ ]:
# How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name followed by the facility name.

SELECT DISTINCT CONCAT(m.firstname, ' ', m.surname) AS member ,f.name AS facility
FROM
cd.members AS m
INNER JOIN cd.bookings AS b
USING(memid)
INNER JOIN cd.facilities AS f
ON b.facid = f.facid 
WHERE f.name LIKE 'Tennis%'
ORDER BY member,facility